# BITS F312 - Neural Network and Fuzzy Logic



# NNFL Assignment 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL/Data_A2/

/content/drive/MyDrive/NNFL/Data_A2


In [ ]:
# listing datasets
%ls -l

total 1234719
-rw------- 1 root root     637638 Oct 31 04:55 Assignment2.pdf
-rw------- 1 root root        259 Oct 31 04:57 class_label.mat
-rw------- 1 root root      40295 Oct 31 04:57 data55.xlsx
-rw------- 1 root root      21269 Oct 31 04:55 data5.xlsx
-rw------- 1 root root 1263647365 Oct 31 04:58 input.mat
drwx------ 2 root root       4096 Nov 18 05:58 logs/


In [ ]:
# libraries required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
# supressing warnings
import warnings
warnings.filterwarnings('ignore')

#Q6

Implement extreme learning machine (ELM) classifier for the classification. You can use Gaussian and
tanh activation functions. Please select the training and test instances using 5-fold cross-validation
technique Evaluate individual accuracy and overall accuracy. The dataset (data5.xlsx) contains 7 features
and the last column is the output (class labels). (Packages such as Scikitlearn, keras, tensorflow, pytorch
etc. are not allowed).

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

df = pd.read_excel('data5.xlsx', header = None)
df = df.sample(frac=1).reset_index(drop=True)
def sigmoidFunction(Z):
    return (1/(1+np.exp(-Z)), Z)

para=[]
def act(x, a, b, act):
    if (act == "gaussian"):
        return np.exp(-b*np.linalg.norm(x-a))
    elif (act == "tanh"):
        num = 1-np.exp(-(np.dot(x.T,a)+b))
        den = 1+np.exp(-(np.dot(x.T,a)+b))
        return (num/den)

def init(hiddenLayer, dimensions):
    a = []
    b = []
    for i in range(hiddenLayer):
        a.append(np.random.rand(dimensions,1))
        b.append(np.random.rand(1))
    return (a,b)

def one_hot(y): 
    onehotencoder = OneHotEncoder() 
    y = onehotencoder.fit_transform(y).toarray()
    return y

def compute(hiddenLayer, train_x, test_x, train_y, test_y, act):

    Y_enc = one_hot(train_y)
    H = np.zeros((train_x.shape[0],hiddenLayer))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i]=act(train_x[j],a[i],b[i],act)
    W = np.dot(np.linalg.pinv(H),Y_enc)
    
    H = np.zeros((test_x.shape[0],hiddenLayer))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i] = act(test_x[j],a[i],b[i],act)

    p = np.dot(H,W)
    p = np.argmax(p,axis=1)
    p = np.reshape(p,newshape=(p.shape[0],1))
    accuracy = test_y-p
    accuracy = np.sum((accuracy!=0).astype(int))
    return (p.shape[0]-accuracy)/p.shape[0]

l = 256


kf = KFold(n_splits = 5)
X = df.iloc[:, 0:7].values
X = (X - np.mean(X, axis=0))/(np.std(X, axis=0))
Y = df.iloc[:,7].values
Y = np.reshape(Y, newshape=(-1,1))
a, b = init(l, X.shape[1])
kf.get_n_splits(X)
fold = 0
acctemp = 0
overall = 0
for train_index, test_index in kf.split(X):
    fold+=1
    training_data_X = X[train_index]
    training_data_Y = Y[train_index]
    testing_data_X = X[test_index]
    testing_data_Y = Y[test_index]
    acctemp = compute(l, training_data_X, testing_data_X, training_data_Y, testing_data_Y, act = "tanh")
    overall+=acctemp
    print("Fold: ", fold, "Accuracy: ", acctemp)
print("Overall Accuracy (tanh) : ", overall/5, '\n')

kf = KFold(n_splits=5)
X=df.iloc[:,0:7].values
X=(X-np.mean(X,axis=0))/(np.std(X,axis=0))
Y=df.iloc[:,7].values
Y=np.reshape(Y,newshape=(-1,1))
a,b=init(l,X.shape[1])
kf.get_n_splits(X)
fold=0
accuracy=0
overall=0
for train_index, test_index in kf.split(X):
    fold+=1
    training_data_X = X[train_index]
    training_data_Y = Y[train_index]
    testing_data_X = X[test_index]
    testing_data_Y = Y[test_index]
    accuracy = compute(l,training_data_X,testing_data_X,training_data_Y,testing_data_Y,"gaussian")
    overall+= accuracy
    print("Fold: ",fold," Accuracy: ", accuracy)
print("Overall Accuracy (Gaussian) : " , overall/5)


Fold: 1 Accuracy: 0.913953488372093
Fold: 2 Accuracy: 0.8930232558139535
Fold: 3 Accuracy: 0.8853146853146853
Fold: 4 Accuracy: 0.8930232558139535
Fold: 5 Accuracy: 0.8202797202797203
Overall Accuracy (tanh) : 0.8011188811188812 

Fold: 1 Accuracy: 0.937213488372093
Fold: 2 Accuracy: 0.9122232558139535
Fold: 3 Accuracy: 0.9255146853146853
Fold: 4 Accuracy: 0.8648d32558139535
Fold: 5 Accuracy: 0.9234497202797203
Overall Accuracy (Gaussian) : 0.9067688811188812
